<a href="https://colab.research.google.com/github/abhijeetamle/Deep-Learning/blob/master/mnist_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#specifying tensorflow version 1.x for 1.15.0
%tensorflow_version 1.x
#import tensorflow
import tensorflow as tf
#print(tf.__version__)
#import MNIST dataset handle from tensorflow and same data in one hot encoded format
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

**To check whether MNIST dataset got loaded or not**

running the below cell will show few images with their labels from the dataset

In [0]:
data = mnist.train.next_batch(5)
images = data[0]
labels = data[1]

# import matplotlib for visualization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

for index, image in enumerate(images):
    print( 'Label:', labels[index])
    print( 'Digit in the image', np.argmax(labels[index]))
    plt.imshow(image.reshape(28,28),cmap='gray')
    plt.show()

In [2]:
# defining placeholder
# x is a input with 28x28 pixels = 784
x = tf.placeholder(tf.float32, shape=[None, 784])
print('x.get_shape(): {}'.format(x.get_shape()))

# y is an output with 10 digits
y = tf.placeholder(tf.float32, shape=[None, 10])

# Reshaping the array to 4-dims so that it can work with TensorFlow convolution and pooling functions
input_tensor = tf.reshape(x, [-1,28,28,1])
print('input_tensor.get_shape(): {}'.format(input_tensor.get_shape()))

x.get_shape(): (?, 784)
input_tensor.get_shape(): (?, 28, 28, 1)


**Function to create convolution layer**


In [0]:
# Function to create convolution layer then performing max pool with stride of 2x2
def create_convolution_layer(input_tensor, num_input_channels, num_filters, kernel, stride, name):
    print('Name: {} -- input_tensor.get_shape(): {}'.format(name,input_tensor.get_shape()))
    # for tf.nn.conv2d create the input shape filter
    conv_filter_shape = [kernel[0], kernel[1], num_input_channels, num_filters]
    print('Name: {} -- conv_filter_shape: {}'.format(name,conv_filter_shape))

    # creating the weights
    weight = tf.Variable(tf.truncated_normal(conv_filter_shape,stddev=0.03), name=name+'_weight')
    print('Name: {} -- weight: {}'.format(name,weight))

    # creating the bias
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_bias')
    print('Name: {} -- bias: {}'.format(name,bias))

    # setup the convolutional layer operation using weights
    con = tf.nn.conv2d(input_tensor, weight, strides=[1, 1, 1, 1], padding='SAME')
    
    # Define activation operation: add a bias to the output of the convolutional filter, then apply a ReLU non-linear activation function
    activation = tf.nn.relu(tf.nn.bias_add(con, bias))

    # performing max pooling on activation
    # here we are using a 2×2 max pooling window size

    activation = tf.nn.max_pool(value=activation,
                                ksize=[1, stride[0], stride[1], 1],
                                strides=[1, 2, 2, 1],
                                padding='SAME')
    
    return activation

In [0]:
# we have to flatten the output from the last convolutional layer to 1D array

def flatten(x):    
    
    input_shape=x.get_shape()
    flatten_num = 1
    for index in range(len(input_shape)-1):
        flatten_num *= input_shape[1-len(input_shape)+index].value
    x = tf.reshape(x, [-1,flatten_num])

    print('flatten: {}'.format(x.get_shape()))
    return x

#    print(x.get_shape())


**Function to create fully connected layer**

In [0]:
def create_fully_connected_layer(x, num_output, activation_fun=None, name=None):

    # creating the weights
    weights = tf.Variable(tf.truncated_normal([x.get_shape()[-1].value, num_output], stddev=0.03), name=name+'_weight')
    print('Name: {} -- weight: {}'.format(name,weights))

    # creating the bias
    bias = tf.Variable(tf.truncated_normal([num_output]), name=name+'_bias')

    # Calculate sum of neurons
    sum = tf.matmul(x,weights) + bias
        
        # applying activation function if required
    if activation_fun:
      sum = activation_fun(sum)

    return sum

**Designing the network**

In [6]:
# Creating convolution layers

    # layer 1: 5x5 conv, 1 input, 32 outputs
      #   name = layer1
      #   input_tensor = input_tensor
      #   num_input_channels = 1
      #   num_filters = 32
      #   kernel = [5,5]
      #   stride = [2,2]

conv_layer1 = create_convolution_layer(input_tensor, 1, 32, [5, 5], [2, 2],name='conv_layer1')

    # layer 2: 5x5 conv, 32 inputs, 64 outputs
      #   name = layer2
      #   input_tensor = layer1
      #   num_input_channels = 32
      #   num_filters = 64
      #   kernel = [5,5]
      #   stride = [2,2]
  
conv_layer2 = create_convolution_layer(conv_layer1, 32, 64, [5,5], [2,2],name='conv_layer2')

# Flattening output to 1D array

flatten_output = flatten(conv_layer2)

# Creating fully connected dense layers

    # layer 1: number of hidden layer nodes = 3136

dense_layer1 = create_fully_connected_layer(flatten_output, 1024, tf.nn.relu, name='dense_layer1')

    #drop = tf.nn.dropout(dense_layer1)

    # layer 2: number of hidden layer nodes = 1024

last_layer = create_fully_connected_layer(dense_layer1, 10, tf.nn.softmax, name='dense_layer2')

Name: conv_layer1 -- input_tensor.get_shape(): (?, 28, 28, 1)
Name: conv_layer1 -- conv_filter_shape: [5, 5, 1, 32]
Name: conv_layer1 -- weight: <tf.Variable 'conv_layer1_weight:0' shape=(5, 5, 1, 32) dtype=float32_ref>
Name: conv_layer1 -- bias: <tf.Variable 'conv_layer1_bias:0' shape=(32,) dtype=float32_ref>
Name: conv_layer2 -- input_tensor.get_shape(): (?, 14, 14, 32)
Name: conv_layer2 -- conv_filter_shape: [5, 5, 32, 64]
Name: conv_layer2 -- weight: <tf.Variable 'conv_layer2_weight:0' shape=(5, 5, 32, 64) dtype=float32_ref>
Name: conv_layer2 -- bias: <tf.Variable 'conv_layer2_bias:0' shape=(64,) dtype=float32_ref>
flatten: (?, 3136)
Name: dense_layer1 -- weight: <tf.Variable 'dense_layer1_weight:0' shape=(3136, 1024) dtype=float32_ref>
Name: dense_layer2 -- weight: <tf.Variable 'dense_layer2_weight:0' shape=(1024, 10) dtype=float32_ref>


In [7]:
print(conv_layer1.get_shape())
print(conv_layer2.get_shape())
print(dense_layer1.get_shape())
print(last_layer.get_shape())

(?, 14, 14, 32)
(?, 7, 7, 64)
(?, 1024)
(?, 10)


**Define loss and optimizer**

In [9]:
# Defining loss

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=last_layer, labels=y))

# Adding an optimizer

optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)

# Defining accuracy

correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(last_layer, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Adding a summary for storing loss and accuracy

tf.summary.scalar('loss', loss)
tf.summary.scalar('accuracy',accuracy)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



<tf.Tensor 'accuracy:0' shape=() dtype=string>

**Creating a Tensorflow session**

In [0]:
#create a tensorflow session
sess = tf.InteractiveSession()

summary_log = tf.summary.merge_all()

In [11]:
batch_size = 50
total_batch_size = int(len(mnist.train.labels) / batch_size)
print('total mnist training labels: {}'.format(len(mnist.train.labels)))
print('total_batch_size: {}'.format(total_batch_size))

total mnist training labels: 55000
total_batch_size: 1100


**Training the network**

In [0]:
# Start time of training

import time
start_time = time.time()

# initializing the variables

sess.run(tf.global_variables_initializer())

# for each epoch running the traning of network in batch of 50

for epoch in range(10):

  print('Starting epoch: {}'.format(epoch+1))

  for batch_ctr in range(total_batch_size):

    # Fetching a batch size of 50 images
    batch = mnist.train.next_batch(50)
  
    # Every 100 steps, compute & print the accuracy of the network's prediction
    if batch_ctr % 100 == 0:
      train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y: batch[1]})
      print('step %d, training accuracy %g' % (batch_ctr, train_accuracy))
      #print('currently going: {}'.format(batch_ctr))

    # run training
    sess.run(optimizer, feed_dict={x: batch[0], y: batch[1]})


print('Total traning time: {}'.format(time.time() - start_time))